<a href="https://colab.research.google.com/github/linusgun/PythonCourses/blob/main/New_Tokyo_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade plotly

     |████████████████████████████████| 27.7 MB 1.6 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


In [ ]:
!mkdir ~/.kaggle #create a directory
!cp kaggle.json ~/.kaggle/ #copy the file in the directory
!chmod 600 ~/.kaggle/kaggle.json #change the permissions of the file
!kaggle datasets download -d arjunprasadsarkhel/2021-olympics-in-tokyo #download the file
!unzip 2021-olympics-in-tokyo.zip #unzip de file

  0% 0.00/348k [00:00<?, ?B/s]
100% 348k/348k [00:00<00:00, 65.0MB/s]
Archive:  2021-olympics-in-tokyo.zip
  inflating: Athletes.xlsx           
  inflating: Coaches.xlsx            
  inflating: EntriesGender.xlsx      
  inflating: Medals.xlsx             
  inflating: Teams.xlsx              


In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 50.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=cc2ab6f9875d298252b2e0f9298f070626b111545b25f16de226c510b8aec8ba
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import pyspark
spark= pyspark.sql.SparkSession(pyspark.SparkContext())
spark

In [ ]:
import pandas as pd

def excel_to_csv(name):
  pd.read_excel(name + '.xlsx').to_csv(name + '.csv',sep=',',index=False)

excel_to_csv('Athletes')
excel_to_csv('Coaches')
excel_to_csv('EntriesGender')
excel_to_csv('Medals')
excel_to_csv('Teams')


/usr/local/lib/python3.7/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
athletes=pd.read_csv('Athletes.csv')
coaches=pd.read_csv('Coaches.csv')
entries_gender=pd.read_csv('EntriesGender.csv')
medals=pd.read_csv('Medals.csv')
teams=pd.read_csv('Teams.csv')

In [ ]:
medals.head()
medals.sort_values('Rank by Total',inplace=True)
medals.head()


,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
4,5,ROC,20,28,23,71,3
3,4,Great Britain,22,21,22,65,4
2,3,Japan,27,14,17,58,5


In [ ]:
import plotly.express as px
Fig=px.bar(medals,x='Team/NOC',y=['Gold', 'Silver', 'Bronze'], color_discrete_sequence=['Gold', 'Silver', 'Brown'], title='Number of Medals per Country')
Fig.show()

In [ ]:
spark.read.csv('Coaches.csv',header=True,inferSchema=True).createOrReplaceTempView('Coaches') #case sensitive createorReplaceTempView
spark.sql('select * from coaches').show(7)


+--------------------+-------------+----------+-----+
|                Name|          NOC|Discipline|Event|
+--------------------+-------------+----------+-----+
|     ABDELMAGID Wael|        Egypt|  Football| null|
|           ABE Junya|        Japan|Volleyball| null|
|       ABE Katsuhiko|        Japan|Basketball| null|
|        ADAMA Cherif|Côte d'Ivoire|  Football| null|
|          AGEBA Yuya|        Japan|Volleyball| null|
|AIKMAN Siegfried ...|        Japan|    Hockey|  Men|
|       AL SAADI Kais|      Germany|    Hockey|  Men|
+--------------------+-------------+----------+-----+
only showing top 7 rows



In [ ]:
query_coaches=spark.sql('select NOC, count(NOC) as coaches from coaches group by NOC order by NOC')
#query_coaches.show(10)

pd_query_coaches=query_coaches.toPandas()
#pd_query_coaches.head(10)

Fig = px.bar(pd_query_coaches, x='NOC',y='coaches',color='coaches', title='Number of Coaches per Country')
Fig.show()

In [ ]:
athletes.head(5)

,Name,NOC,Discipline
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball


In [ ]:
spark.read.csv('Athletes.csv',header=True,inferSchema=True).createOrReplaceTempView('Athletes')
query_athletes=spark.sql('''select first(NOC) as NOC, first(Discipline) as Discipline,
                            count(*) as count from Athletes group by NOC, Discipline 
                            order by NOC, Discipline''')
query_athletes.show(10)
                          

+-----------+-------------------+-----+
|        NOC|         Discipline|count|
+-----------+-------------------+-----+
|Afghanistan|          Athletics|    2|
|Afghanistan|           Shooting|    1|
|Afghanistan|           Swimming|    1|
|Afghanistan|          Taekwondo|    1|
|    Albania|Artistic Gymnastics|    1|
|    Albania|          Athletics|    1|
|    Albania|               Judo|    1|
|    Albania|           Shooting|    1|
|    Albania|           Swimming|    2|
|    Albania|      Weightlifting|    2|
+-----------+-------------------+-----+
only showing top 10 rows



In [ ]:
pd_query_athletes=query_athletes.toPandas()

Fig=px.bar(pd_query_athletes,x='NOC',y='count',color='Discipline', title='Number of athltes for each country per discipline')
Fig.show()

In [ ]:
athletes['Discipline'].value_counts().head(5)

Athletics    2068
Swimming      743
Football      567
Rowing        496
Hockey        406
Name: Discipline, dtype: int64

In [ ]:
pd_query_athletes.head(10)
pd_query_athletes = pd_query_athletes.loc[pd_query_athletes['Discipline'].isin(['Athletics','Swimming','Football'])]
pd_query_athletes.Discipline.value_counts()

Athletics    195
Swimming     181
Football      24
Name: Discipline, dtype: int64

In [ ]:
Fig=px.bar(pd_query_athletes,x='NOC',y='count',color='Discipline', title='Number of athletes for each country per discipline')
Fig.show()

In [ ]:
entries_gender.head(5)

,Discipline,Female,Male,Total
0,3x3 Basketball,32,32,64
1,Archery,64,64,128
2,Artistic Gymnastics,98,98,196
3,Artistic Swimming,105,0,105
4,Athletics,969,1072,2041


In [ ]:
len(entries_gender)

46

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

specs = [[{'type':'domain'},{'type':'domain'}]]*23

fig= make_subplots(rows=23, cols=2, 
                   subplot_titles=entries_gender['Discipline'], specs=specs)

sub_figs=[]
for index, row in entries_gender.iterrows():
  sub_fig = go.Pie(labels=['Female', 'Male'], values=[row['Female'],
                                                      row['Male']])
  sub_figs.append(sub_fig)

k=0
for i in range(1,24):
  for j in range(1,3):
    fig.add_trace(sub_figs[k],i,j)
    k+=1

fig.update_layout(showlegend=False, height=10000, width=800, title_text='Distribution of Gender amongst each Discipline')
fig.update_traces(textposition='inside', textinfo='label+percent', hoverinfo='label+value+percent')

fig=go.Figure(fig)
fig.show()
